# Prétraitement des données

## Suppression des éléments indésirables de nos tweets

In [1]:
# Imports nécessaires
import pandas as pd
import numpy as np
import string 
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from nltk.stem import WordNetLemmatizer

In [25]:
# Lecture et stockage de la base de données
tweet_df = pd.read_pickle('../../delphes/data/final4_clean.csv')
tweet_df.head()
new_test_df = pd.read_pickle('../../delphes/data/extended_tweet_df')

In [26]:
y = np.arange(0, tweet_df.shape[0])
tweet_df['index'] = y
tweet_df.set_index('index', inplace=True)

In [27]:
# Remove the undesirable elements in the entire dataframe
def rmurl_df(df, column_name):
    '''
    This function removes all the URLs, the #hashtag and the @user of a column made of strings.
    Be careful to apply it BEFORE all the other preprocessing steps (if not it wont'
    be recognized as a URL)
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)
    return df



In [28]:
# Lowercase the tweet's column
def lower_df(df, column_name):
    '''
    This function lowercases a column made of strings.
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.lower()
    return df

In [29]:
# Remove the numbers in the tweet's column
def rmnumbers_df(df, column_name):
    '''
    This function removes all the digits of a column made of strings.
    '''
    df = df.copy()
    def remove_numbers(text):
        return ''.join(word for word in text if not word.isdigit())
    df[column_name] = df[column_name].apply(remove_numbers)
    return df

In [30]:
# Remove the undesirable punctuations in the tweet's column
def rmpunct_df(df, column_name):
    '''
    This function removes all the punctuations, all the "rt" and remove multiple spaces
    of a column made of strings.
    '''
    punct = string.punctuation
    df = df.copy()
    def replace_punct(text):
        for punctu in punct:
            text = text.replace(punctu, ' ')
        text = text.replace(' rt ','')
        text = " ".join(text.split())
        return text
    df[column_name] = df[column_name].apply(replace_punct)
    return df

In [31]:
# Remove the stopwords in the tweet's column
def rmstopwords_df(df, column_name):
    '''
    This function removes all the stopwords of a column made of strings.
    '''
    df = df.copy()
    stop_words = stopwords.words('english')
    def remove_stopwords(text):
        for word in stop_words:
            text = text.replace(f' {word} ', ' ')
        return text
    df[column_name] = df[column_name].apply(remove_stopwords)
    return df

In [32]:
# Lemmatize a column in a dataset
def lemmatize_df(df, column_name):
    '''
    This function lemmatize the words of a column made of strings.
    '''
    df = df.copy()
    def lemmatize(text):
        lemmatizer = WordNetLemmatizer()
        retour = []
        for word in text:
            retour.append(lemmatizer.lemmatize(word))
        text = ''.join(word for word in retour)
        return text

    df[column_name] = df[column_name].apply(lemmatize)
    return df

In [33]:
# Erase all the words that are 1-letter or 2-letters long
def erase_fewletter_df(df, column_name):
    '''
    One or two letters words are deleted from the dataset.
    '''
    df = df.copy()
    def tester(text):
        text = ' '.join( [w for w in text.split() if len(w)>2] )
        return text

    df[column_name] = df[column_name].apply(tester)
    return df


In [34]:
# Remove the undesirable emojis in the entire dataframe
def rmemojis_df(df):
    '''
    This function removes all the emojis of a column made of strings.
    Be careful to translate in latin alphabet before applying this function : 
    it also removes cyrillic alphabet
    '''
    df = df.copy()
    df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    return df

In [35]:
tweet_df

,mep_id,name,country,group,nat_group,twitter,content
index,,,,,,,
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,[@danutahuebner Bardzo dziękuję @danutahuebner...
1,189525,Asim ADEMOV,Bulgaria,Group of the European People's Party (Christia...,Citizens for European Development of Bulgaria,AdemovAsim,[135 години единна и силна България. \nЧестит ...
2,124831,Isabella ADINOLFI,Italy,Non-attached Members,Movimento 5 Stelle,Isa_Adinolfi,"[Sembra un film, ma purtroppo è realtà: le imm..."
3,125045,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,Partido Socialista Obrero Español,ClaraAguilera7,"[Clara Aguilera: ""El criterio científico debe ..."
4,204335,Alviina ALAMETSÄ,Finland,Group of the Greens/European Free Alliance,Vihreä liitto,alviinaalametsa,[@LHurttila @MariaOhisalo Kaupungilla on järje...
...,...,...,...,...,...,...,...
470,124884,Kosma ZŁOTOWSKI,Poland,European Conservatives and Reformists Group,Prawo i Sprawiedliwość,KosmaZlotowski,"[https://t.co/7NnVzDXmis, Dziś inauguracja!\nh..."
471,197621,Juan Ignacio ZOIDO ÁLVAREZ,Spain,Group of the European People's Party (Christia...,Partido Popular,zoidoji,[Desde @ppegrupo reclamamos una mayor segurida...
472,124739,Carlos ZORRINHO,Portugal,Group of the Progressive Alliance of Socialist...,Partido Socialista,czorrinho,[@sofiaferreira57 O meu mail profissional é pú...


In [36]:
clean_df = rmurl_df(new_test_df, 'content')
clean_df = lower_df(clean_df, 'content')
clean_df = rmnumbers_df(clean_df, 'content')
clean_df = rmpunct_df(clean_df, 'content')
clean_df = rmstopwords_df(clean_df, 'content')
clean_df = lemmatize_df(clean_df, 'content')
clean_df = erase_fewletter_df(clean_df, 'content')
clean_df = rmemojis_df(clean_df)

In [37]:
clean_df.to_pickle('BDD smooth')

In [38]:
clean_df

,mep_id,name,country,group,nat_group,twitter,content
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,bardzo dzikuj niezalene wadzy wolne media daj ...
1,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,komisja przyja arcywane projekty pilotaowe aut...
2,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,komisja przyja projekty pilotaowe mojego wspau...
3,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,tym dniu tym miejscu tej godzinie prosz jedno ...
4,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,bg nie potrzebuje by przez nikogo broniony nie...
...,...,...,...,...,...,...,...
137295,96933,Milan ZVER,Slovenia,Group of the European People's Party (Christia...,Slovenska demokratska stranka,MilanZver,tedenska akcija etrtka srede zbornik bela knji...
137296,96933,Milan ZVER,Slovenia,Group of the European People's Party (Christia...,Slovenska demokratska stranka,MilanZver,strong amp way forward european future read eu...
137297,96933,Milan ZVER,Slovenia,Group of the European People's Party (Christia...,Slovenska demokratska stranka,MilanZver,date noted worrying state danger ends special ...
137298,96933,Milan ZVER,Slovenia,Group of the European People's Party (Christia...,Slovenska demokratska stranka,MilanZver,vsi vedo nekaj noro strahu pred nasilnimi levi...


In [39]:
testbase_df = clean_df[clean_df['country'] == 'Ireland']
testbase_deputy_df = tweet_df[tweet_df['country'] == 'Ireland']

## Création du modèle

### Traitement des données d'entraînement et de test

In [53]:
# Cette fonction retourne automatiquement X_train, X_test, y_train, y_test de notre base de données twitter.
def get_train_test_objects(df):
    '''
    Les étapes que cette fonction réalise sont en commentaires.
    '''
    # Copie de la base de données pour éviter les problèmes d'assignation abusive.
    df = df.copy() 
    # Récupération de tous les tweets et du nom du député qui les a posté. Création de la cible y.
    df = df[['name', 'content']]
    y = pd.get_dummies(df['name'])
    # Transformation des tweets en suite de mots (strings) dans une liste.
    sentences = df['content']
    sentences_inter = []
    for sentence in sentences:
        sentences_inter.append(sentence.split())
    # Séparation des données d'entraînement et de test
    sentences_train, sentences_test, y_train, y_test = train_test_split(sentences_inter, y, test_size = 0.3)
    # Vectorisation des phrases
    word2vec = Word2Vec(sentences=sentences_train)
    # Création des données d'entrée.
    X_train = embedding(word2vec,sentences_train)
    X_test = embedding(word2vec,sentences_test)
    X_train_pad = pad_sequences(X_train, padding='post',value=-1000, dtype='float32')
    X_test_pad = pad_sequences(X_test, padding='post',value=-1000, dtype='float32')
    # Création des données cibles.
    y_train = y_train.values
    y_test = y_test.values
    # Sorties de la fonction
    return X_train_pad, y_train, X_test_pad, y_test, word2vec

In [54]:
def embed_sentence(word2vec, sentence):
    y = []
    for word in sentence:
        if word in word2vec.wv.vocab.keys():
           y.append(word2vec[word])
    return np.array(y)

def embedding(word2vec, sentences):
    
    y = []
    for sentence in sentences:
        y.append(embed_sentence(word2vec, sentence))
    return y

In [56]:
X_train, y_train, X_test, y_test, word2vec = get_train_test_objects(testbase_df)

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [57]:
X_train.shape

(2100, 27, 100)

### Initiation du modèle

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = -1000))
    model.add(layers.LSTM(13, activation='tanh'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
    return model


### Entraînement du modèle et évaluation

In [45]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 100, monitor='val_loss', restore_best_weights=True)
model = init_model()
model.fit(X_train, y_train, batch_size = 8, epochs=1000, validation_split = 0.3, callbacks = [es])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1470 samples, validate on 630 samples
Epoch 1/1000
1470/1470 [==============================] - 8s 6ms/sample - loss: 2.3025 - acc: 0.1061 - val_loss: 2.3024 - val_acc: 0.0889
Epoch 2/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.2993 - acc: 0.1034 - val_loss: 2.2971 - val_acc: 0.0905
Epoch 3/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.2879 - acc: 0.1252 - val_loss: 2.2920 - val_acc: 0.1540
Epoch 4/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.2735 - acc: 0.1381 - val_loss: 2.2675 - val_acc: 0.1238
Epoch 5/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.2617 - acc: 0.1435 - val_loss: 2.2535 - val_acc: 0.1381
Epoch 6/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.2527 -

Epoch 55/1000
1470/1470 [==============================] - 7s 5ms/sample - loss: 2.1492 - acc: 0.2102 - val_loss: 2.2284 - val_acc: 0.1841
Epoch 56/1000
1470/1470 [==============================] - 8s 5ms/sample - loss: 2.1510 - acc: 0.2027 - val_loss: 2.2547 - val_acc: 0.1714
Epoch 57/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.1466 - acc: 0.2163 - val_loss: 2.2573 - val_acc: 0.1841
Epoch 58/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.1480 - acc: 0.2054 - val_loss: 2.2201 - val_acc: 0.1746
Epoch 59/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.1446 - acc: 0.2048 - val_loss: 2.2430 - val_acc: 0.1651
Epoch 60/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.1484 - acc: 0.2014 - val_loss: 2.2225 - val_acc: 0.1810
Epoch 61/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.1407 - acc: 0.2136 - val_loss: 2.2437 - val_acc: 0.1698 loss: 2.1440 - ac
Epoch 62/

Epoch 114/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.0899 - acc: 0.2333 - val_loss: 2.2601 - val_acc: 0.1762
Epoch 115/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.0861 - acc: 0.2395 - val_loss: 2.2712 - val_acc: 0.1778
Epoch 116/1000
1470/1470 [==============================] - 6s 4ms/sample - loss: 2.0879 - acc: 0.2293 - val_loss: 2.3003 - val_acc: 0.1794
Epoch 117/1000
1470/1470 [==============================] - 7s 5ms/sample - loss: 2.0894 - acc: 0.2340 - val_loss: 2.2726 - val_acc: 0.1794
Epoch 118/1000
1470/1470 [==============================] - 8s 5ms/sample - loss: 2.0880 - acc: 0.2333 - val_loss: 2.2890 - val_acc: 0.1873
Epoch 119/1000
1470/1470 [==============================] - 8s 5ms/sample - loss: 2.0847 - acc: 0.2320 - val_loss: 2.2643 - val_acc: 0.1714
Epoch 120/1000
1470/1470 [==============================] - 7s 4ms/sample - loss: 2.0907 - acc: 0.2442 - val_loss: 2.2782 - val_acc: 0.1683
Epoch 121/1000
1470/

In [59]:
model.evaluate(X_test, y_test)

900/900 [==============================] - 0s 122us/sample - loss: 2.2132 - acc: 0.1800


[2.213210598627726, 0.18]

In [60]:
# Renvoie le député le plus proche de votre tweet
def predict_deputy(df, model, tweet, by_tweet = False):
    '''
    La fonction prend la base de données originale (par député), un modèle entraîné et un texte en entrée.
    Elle renvoie le député le plus proche du texte proposé.
    Attention : le texte en entrée doit être une liste d'au moins deux éléments (strings).
    Quand by_tweet = False, on ressort le député le plus proche de l'ENSEMBLE des tweets.
    Quand by_tweet = True, on sort le député le plus proche POUR CHAQUE tweet.
    '''
    tweet_inter = []
    for tw in tweet:
        tweet_inter.append(tw.split())
    X_example = embedding(word2vec,tweet_inter)
    X_example_pad = pad_sequences(X_example, padding='post',value=-1000, dtype='float32')
    prediction = model.predict(X_example_pad)
    if not by_tweet:
        deputy = list(df['name'])[prediction.sum(axis=0).argmax()]
        return deputy
    else:
        deputies_by_tweet = []
        for element in prediction:
            deputies_by_tweet.append(list(df['name'])[element.argmax()])
        return deputies_by_tweet

In [61]:
prediction = predict_deputy(testbase_deputy_df,
                            model, 
                            ['the northern ireland protocol must protected line real effort must made earnest week bridge gap talks come table many times good faith amp view securing future partnership respects standards union', 
                             'member special committee beating cancer look forward working members contributing important fight cancer',
                             'this month shining light childrens cancer parliament buildings lit gold highlight survivors childrens cancer well diagnosed cancer'], 
                            by_tweet=False)

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [62]:
prediction

'Luke Ming FLANAGAN'

In [63]:
testbase_df['content'][24724]

'this month shining light childrens cancer parliament buildings lit gold highlight survivors childrens cancer well diagnosed cancer'

In [64]:
testbase_df

,mep_id,name,country,group,nat_group,twitter,content
24722,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,the northern ireland protocol must protected l...
24723,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,member special committee beating cancer look f...
24724,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this month shining light childrens cancer parl...
24725,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,yesterday told europe need keep speed amp infr...
24726,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this letter sent taoiseach clear seeking clari...
...,...,...,...,...,...,...,...
130490,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,brendan its favourite item desk
130491,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,monday mornings mean tidy desk chaos ahead wee...
130492,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,fergal post kick new week thank folks like mak...
130493,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,it took rare vision unique leadership see beyo...


# Bac à sable

In [65]:
danish_df = pd.read_pickle('../../delphes/data/merger')

In [66]:
working_df = danish_df[['name','content_y']].dropna().reset_index()

In [67]:
working_df

,index,name,content_y
0,0,Margrete AUKEN,"[False, [0.101227, -0.00355242, -0.0613298, 0...."
1,2,Margrete AUKEN,"[[-0.0540806, 0.0168135, 0.0596281, 0.0681775,..."
2,3,Margrete AUKEN,"[[0.00612917, 0.0156791, -0.0486738, 0.0480299..."
3,4,Margrete AUKEN,"[False, [-0.0484475, 0.011017, -0.0737066, 0.0..."
4,5,Margrete AUKEN,"[[-0.0989527, 0.0149615, 0.0241871, 0.0582589,..."
...,...,...,...
11178,11225,Lara WOLTERS,"[[0.0495577, 0.129444, -0.0491547, 0.00429166,..."
11179,11226,Lara WOLTERS,[False]
11180,11227,Lara WOLTERS,"[[-0.0972181, -0.0736294, 0.00087396, 0.063221..."
11181,11228,Lara WOLTERS,"[[-0.0196253, -0.00134424, -0.035015, -0.01173..."


In [68]:
working_df['content_y'].shape[0]

11183

In [69]:
type(working_df['content_y'][0][4])

numpy.ndarray

In [70]:
new_items = [x if type(x) == np.ndarray else np.array([-1000]*100) for x in working_df['content_y'][4]]

In [71]:
new_items

[array([-0.0989527 ,  0.0149615 ,  0.0241871 ,  0.0582589 , -0.0471224 ,
        -0.0545909 ,  0.00883459, -0.0963067 , -0.0200379 ,  0.0645213 ,
         0.00059959, -0.0359953 ,  0.0905238 , -0.019637  , -0.119255  ,
        -0.0085331 , -0.0910109 , -0.0317265 ,  0.0437045 ,  0.042118  ,
        -0.0413337 ,  0.0793529 , -0.0958057 , -0.0859956 , -0.070275  ,
         0.0277338 ,  0.0245911 , -0.0532877 ,  0.0901591 , -0.0341724 ,
        -0.0523973 ,  0.06913   , -0.0358292 ,  0.0490628 , -0.0533377 ,
        -0.0590526 , -0.018908  , -0.0680928 ,  0.0428848 ,  0.00446642,
         0.0254242 ,  0.029823  , -0.022389  , -0.0298494 , -0.00220434,
         0.0236405 , -0.0202339 ,  0.071565  ,  0.00653942,  0.04787   ,
         0.0241135 , -0.00861419, -0.072192  , -0.0061101 , -0.0421144 ,
        -0.0441334 ,  0.0284948 ,  0.0101265 ,  0.0494711 ,  0.123695  ,
         0.00318411, -0.147268  ,  0.0619475 , -0.0521131 , -0.0270893 ,
        -0.183071  , -0.121067  , -0.0656926 , -0.0

In [72]:
for i in range(working_df['content_y'].shape[0]):
    working_df['content_y'][i] = [x if type(x) == np.ndarray else np.array([-1000.5]*300) for x in working_df['content_y'][i]]

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
working_df = working_df[['name', 'content_y']]
working_df.rename(columns={"content_y": "content"}, inplace=True)

c:\users\psmag\.venvs\delphes\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [74]:
X = working_df[['content']]

In [208]:
y = pd.get_dummies(working_df['name'])
X_train2, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
# Création des données cibles.
y_train = y_train.values
y_test = y_test.values

In [207]:
def treating_X(X):
    X = X.reset_index().drop('index', axis=1)
    maxi = 0
    for i in range(len(X['content'])):
        if len(X['content'][i]) > maxi:
            maxi = len(X['content'][i])
    c = np.zeros((X.shape[0], maxi, 300))
    i = 0
    for element in X['content']:
        a_len = len(element)
        a = np.array(element)
        c[i][:a_len] = a
        i+=1
    return c

In [209]:
c = treating_X(X_train2)

In [305]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = 0))
    model.add(layers.LSTM(150, dropout=0.3, recurrent_dropout=0.3, activation='tanh'))
    model.add(layers.Dense(128, activation='tanh'))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
    return model

In [306]:
c.shape

(7828, 57, 300)

In [307]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [308]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 10, monitor='val_loss', restore_best_weights=True)
model = init_model()
model.fit(c, y_train, batch_size = 8, epochs=1000, validation_split = 0.3, callbacks = [es])

Train on 5479 samples, validate on 2349 samples
Epoch 1/1000
5479/5479 [==============================] - 243s 44ms/sample - loss: 3.4488 - acc: 0.0485 - val_loss: 3.2229 - val_acc: 0.0664
Epoch 2/1000
5479/5479 [==============================] - 249s 45ms/sample - loss: 3.2154 - acc: 0.0783 - val_loss: 3.1360 - val_acc: 0.0830
Epoch 3/1000
5479/5479 [==============================] - 251s 46ms/sample - loss: 3.1465 - acc: 0.0854 - val_loss: 3.1272 - val_acc: 0.0754
Epoch 4/1000
5479/5479 [==============================] - 253s 46ms/sample - loss: 3.1114 - acc: 0.0986 - val_loss: 3.0512 - val_acc: 0.1056
Epoch 5/1000
5479/5479 [==============================] - 255s 46ms/sample - loss: 3.0808 - acc: 0.1048 - val_loss: 3.0412 - val_acc: 0.1026
Epoch 6/1000
5479/5479 [==============================] - 258s 47ms/sample - loss: 3.0275 - acc: 0.1247 - val_loss: 3.0140 - val_acc: 0.1145
Epoch 7/1000
5479/5479 [==============================] - 259s 47ms/sample - loss: 2.9861 - acc: 0.1289 - 

KeyboardInterrupt: 

In [304]:
model.evaluate(c_test, y_test)

3355/3355 [==============================] - 8s 2ms/sample - loss: 2.9406 - acc: 0.1556


[2.9405613182909263, 0.15558867]

### Record à 16.36% avec ce modèle

In [224]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = 0))
    model.add(layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2, activation='tanh'))
    model.add(layers.Dense(128, activation='tanh'))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
    return model

In [299]:
model_record = model